---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [44]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [45]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [46]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = []
    state = None
    state_towns = []
    with open('university_towns.txt') as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            data.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    return df

In [49]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    
    # Value Found
    latest_recession_start = ''
    
    # Clean Yearly Quartely Years
    gpdquarterly_years = df['Unnamed: 4'].dropna().drop([1,2])
    gpdquarterly_years = gpdquarterly_years.reset_index().drop('index', axis=1)
    gpdquarterly_years.columns = ['GDP Quarterly']
    
    # Clean Yearly Quarterly Amounts
    gpdquarterly_amounts = df['Unnamed: 5'].dropna().drop([4])
    gpdquarterly_amounts = gpdquarterly_amounts.reset_index().drop('index', axis=1)
    gpdquarterly_amounts.columns = ['GDP in Billions']
    
    # Merge newly cleaned quarterly dataframes
    new_df = gpdquarterly_years.merge(gpdquarterly_amounts, left_index=True, right_index=True)
    
    # Determine recessions
    for index,value in new_df['GDP in Billions'].items():
        try:
            # Determine the beginning of a reccession
            value_1 = new_df['GDP in Billions'].iloc[index+1]
            value_2 = new_df['GDP in Billions'].iloc[index+2]
            if value_1 < value and value_2 < value_1:
                
                # Ensure that the economy grows after two consecutive declines in gdp
                value_3 = new_df['GDP in Billions'].iloc[index+3]
                value_4 = new_df['GDP in Billions'].iloc[index+4]
                lowest_point = abs(value-value_1-value_2)
                if value_4 > value and value_3 > lowest_point:
                    
                    # Only save the latest recession found after 2000q1
                    recession = new_df['GDP Quarterly'].iloc[index-1]
                    if int(recession.split('q')[0]) > 2000:
                        latest_recession_start = recession
        except IndexError:
            pass
    return latest_recession_start

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    
    # Value Found
    latest_recession_end = ''
    
    # Clean Yearly Quartely Years
    gpdquarterly_years = df['Unnamed: 4'].dropna().drop([1,2])
    gpdquarterly_years = gpdquarterly_years.reset_index().drop('index', axis=1)
    gpdquarterly_years.columns = ['GDP Quarterly']
    
    # Clean Yearly Quarterly Amounts
    gpdquarterly_amounts = df['Unnamed: 5'].dropna().drop([4])
    gpdquarterly_amounts = gpdquarterly_amounts.reset_index().drop('index', axis=1)
    gpdquarterly_amounts.columns = ['GDP in Billions']
    
    # Merge newly cleaned quarterly dataframes
    new_df = gpdquarterly_years.merge(gpdquarterly_amounts, left_index=True, right_index=True)
    
    # Determine recessions
    for index,value in new_df['GDP in Billions'].items():
        try:
            # Determine the beginning of a reccession
            value_1 = new_df['GDP in Billions'].iloc[index+1]
            value_2 = new_df['GDP in Billions'].iloc[index+2]
            if value_1 < value and value_2 < value_1:
                
                # Ensure that the economy grows after two consecutive declines in gdp
                value_3 = new_df['GDP in Billions'].iloc[index+3]
                value_4 = new_df['GDP in Billions'].iloc[index+4]
                lowest_point = abs(value-value_1-value_2)
                if value_4 > value and value_3 > lowest_point:
                    
                    # Only save the latest recession found after 2000q1
                    recession = new_df['GDP Quarterly'].iloc[index+4]
                    if int(recession.split('q')[0]) > 2000:
                        latest_recession_end = recession
        except IndexError:
            pass
    
    return latest_recession_end

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    
    # Value Found
    latest_recession_bottom = ''
    
    # Clean Yearly Quartely Years
    gpdquarterly_years = df['Unnamed: 4'].dropna().drop([1,2])
    gpdquarterly_years = gpdquarterly_years.reset_index().drop('index', axis=1)
    gpdquarterly_years.columns = ['GDP Quarterly']
    
    # Clean Yearly Quarterly Amounts
    gpdquarterly_amounts = df['Unnamed: 5'].dropna().drop([4])
    gpdquarterly_amounts = gpdquarterly_amounts.reset_index().drop('index', axis=1)
    gpdquarterly_amounts.columns = ['GDP in Billions']
    
    # Merge newly cleaned quarterly dataframes
    new_df = gpdquarterly_years.merge(gpdquarterly_amounts, left_index=True, right_index=True)
    
    # Determine recessions
    for index,value in new_df['GDP in Billions'].items():
        try:
            # Determine the beginning of a reccession
            value_1 = new_df['GDP in Billions'].iloc[index+1]
            value_2 = new_df['GDP in Billions'].iloc[index+2]
            if value_1 < value and value_2 < value_1:
                
                # Ensure that the economy grows after two consecutive declines in gdp
                value_3 = new_df['GDP in Billions'].iloc[index+3]
                value_4 = new_df['GDP in Billions'].iloc[index+4]
                lowest_point = abs(value-value_1-value_2)
                if value_4 > value and value_3 > lowest_point:
                    
                    # Only save the latest recession found after 2000q1
                    recession = new_df['GDP Quarterly'].iloc[index+2]
                    if int(recession.split('q')[0]) > 2000:
                        latest_recession_bottom = recession
        except IndexError:
            pass
    
    return latest_recession_bottom

In [53]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv') # Get data
    
    # Keep Quarterly 2000-q1 onward columns
    selected_quarterly_columns = [gdp_monthly for gdp_monthly in df.columns if gdp_monthly.find('-') >= 0 and int(gdp_monthly.split('-')[0]) >= 2000]
    selected_index_columns = ["State","RegionName"]
    
    # Restructure dataframe based on selected columns
    df_gdp = pd.DataFrame(df[selected_quarterly_columns])
    df = pd.DataFrame(df[selected_index_columns])
    
    # Rename column titles to refer quarterly sections per year
    gdp_years = [column.split('-')[0] for column in df_gdp.columns]
    gdp_years = list(dict.fromkeys(gdp_years))
    
    # Take the average of each quartely (every 4 columns)
    df_gdp = df_gdp.groupby((np.arange(len(df_gdp.columns)) // 3) + 1, axis=1).mean()
    
    # Rename columns based on year and its respective quarterly value
    def resetQuarterlyColumns(col=None):
        remainder = int(col)%4
        if remainder >= 0:
            try:
                if remainder != 0:
                    new_gdp = gdp_years[0]+'q'+str(remainder)
                else:
                    new_gdp = gdp_years[0]+'q4'
                    del gdp_years[0]
                return new_gdp
            except IndexError:
                pass
    df_gdp.columns = list(map(resetQuarterlyColumns,[x for x in df_gdp.columns]))
    
    # Join dataframes into one and reset the indeces to 'State' followed by 'RegionName' after converting 'State' to full state names
    df['State'] = [states[state] for state in df['State']]
    df = df.join(df_gdp).set_index(['State','RegionName'])
    
    return df

In [55]:
def run_ttest():
    '''First create new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Get latest recession quarterly statistics
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    universitytowns = get_list_of_university_towns()
    quarterly_housingdata = convert_housing_data_to_quarters()
    
    # Get quarterly non-university housing data
    quarterly_housingdata['Housing Price Rate Ratio'] = quarterly_housingdata[recession_start]-quarterly_housingdata[recession_bottom]
    quarterly_housingdata = quarterly_housingdata[[recession_bottom, recession_start, 'Housing Price Rate Ratio']]
    quarterly_housingdata = quarterly_housingdata.reset_index()
    universitytowns_quarterlyhousingdata = pd.merge(quarterly_housingdata,universitytowns,how='inner',on=['State','RegionName'])
    universitytowns_quarterlyhousingdata['University'] = True
    quarterlyhousingdata = pd.merge(quarterly_housingdata,universitytowns_quarterlyhousingdata,how='outer',on=['State','RegionName',recession_bottom,recession_start,'Housing Price Rate Ratio'])
    quarterlyhousingdata['University'] = quarterlyhousingdata['University'].fillna(False)
    
    # Taking note of towns with or without universities
    towns_universitypresent = quarterlyhousingdata[quarterlyhousingdata['University'] == True]
    towns_universityabsent = quarterlyhousingdata[quarterlyhousingdata['University'] == False]
    
    # Taking a T-test to determine if towns with or without universities have different or similar housing price rate changes
    t,p = ttest_ind(towns_universitypresent['Housing Price Rate Ratio'].dropna(), towns_universityabsent['Housing Price Rate Ratio'].dropna())
    different = True if p<0.01 else False # Different or not (mentioned above)
    better = "university town" if towns_universitypresent['Housing Price Rate Ratio'].mean() < towns_universityabsent['Housing Price Rate Ratio'].mean() else "non-university town" # University or non-university towns are better
        
    return (different, p, better)

run_ttest()

(True, 0.0043252148535112009, 'university town')